In [ ]:
import pyodbc
import pandas as pd
import os
from google.colab import files
from dotenv import load_dotenv  # .env 파일로 환경변수 불러오기

# 1. 환경 변수 불러오기
load_dotenv()

server = os.getenv('DB_SERVER')
database = os.getenv('DB_NAME')
username = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
driver = '{your driver}'
conn_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    conn = pyodbc.connect(conn_string)
    cursor = conn.cursor()
    print("✅ MSSQL 데이터베이스 연결 성공!")
except Exception as e:
    print(f"❌ 데이터베이스 연결 실패: {e}")
    raise

# 2. CSV 파일 업로드
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# 3. NaN 값을 None으로 변환
df = df.where(pd.notnull(df), None)

# 4. 번역 컬럼만 업데이트
updated_rows = 0

for index, row in df.iterrows():
    try:
        cursor.execute("""
            UPDATE notices
            SET content_en = ?, content_zh = ?
            WHERE notice_id = ?
        """,
        row['content_en'],
        row['content_zh'],
        int(row['notice_id']))
        updated_rows += 1
    except Exception as e:
        print(f"❌ 오류 발생 (row {index}, notice_id {row['notice_id']}): {e}")

# 5. 커밋 및 연결 종료
conn.commit()
cursor.close()
conn.close()

print(f"✅ 총 {updated_rows}개의 공지사항 번역이 업데이트되었습니다.")
